In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt_pr
import matplotlib.pyplot as plt_roc
import matplotlib.pyplot as plt_confm
import matplotlib.pyplot as plt_confm_n

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, CondensedNearestNeighbour

from split import build_splits, calculate_se

from sklearn.model_selection import PredefinedSplit, GridSearchCV, StratifiedKFold
from sklearn.metrics import matthews_corrcoef, make_scorer, confusion_matrix, accuracy_score, precision_score, \
    recall_score, balanced_accuracy_score, roc_auc_score, precision_recall_curve, plot_precision_recall_curve, average_precision_score, \
    roc_curve, plot_roc_curve, f1_score, plot_confusion_matrix

In [2]:
embeddings_file = '../data/MSA1_binding_sites.h5'
msa_type = embeddings_file.split("/")[2].split("_")[0]
json_file = '../dataset-analysis/split_21-06-12_18-05_0-9713.json'

X_test, y_test, all_X_train, all_y_train = build_splits(
    json_file,
    # '../data/baseline_embeddings_binding_sites.h5',
    embeddings_file,
    '../data/binding_residues.txt')

Test: 
34473
34473
Split 0: 
25260
25260
Split 1: 
27142
27142
Split 2: 
26950
26950
Split 3: 
26561
26561
Split 4: 
26382
26382


In [13]:
lengths = list(map(len, all_X_train))
total_length = sum(lengths)
folds = np.zeros(shape=total_length)
i = 0
for x, length in enumerate(lengths):
    folds[i:(i+length)] = x
    i += length

old_train_set_size = len(folds)

# transform into 1 big matrix (split into list of vectors??)
big_mac = np.concatenate(all_X_train)
big_mac_y = np.concatenate(all_y_train)

In [ ]:
for ts in range (0, len(lengths)):
    if ts == 0:
        # 1st training split as test
        y_test = big_mac_y[0:lengths[0]]
        X_test = big_mac[0:lengths[0]]
        big_mac = big_mac[lengths[0]:]
        big_mac_y = big_mac_y[lengths[0]:]
        folds = folds[lengths[0]:]
        for i in range(0, len(folds)):
            folds[i] = folds[i] - 1
    elif ts == 1:
        # 2nd training split as test
        y_test = big_mac_y[lengths[0]:(lengths[0]+lengths[1])]
        X_test = big_mac[lengths[0]:(lengths[0]+lengths[1])]
        big_mac = np.concatenate((big_mac[0:lengths[0]], big_mac[(lengths[0]+lengths[1]):]))
        big_mac_y = np.concatenate((big_mac_y[0:lengths[0]], big_mac_y[(lengths[0]+lengths[1]):]))

        folds = np.concatenate((folds[0:lengths[0]], folds[(lengths[0]+lengths[1]):]))
    elif ts == 2:
        # 3rd training split as test
        y_test = big_mac_y[(lengths[0]+lengths[1]):(lengths[0]+lengths[1]+lengths[2])]
        X_test = big_mac[(lengths[0]+lengths[1]):(lengths[0]+lengths[1]+lengths[2])]
        big_mac = np.concatenate((big_mac[0:(lengths[0]+lengths[1])], big_mac[(lengths[0]+lengths[1]+lengths[2]):]))
        big_mac_y = np.concatenate((big_mac_y[0:(lengths[0]+lengths[1])], big_mac_y[(lengths[0]+lengths[1]+lengths[2]):]))

        folds = np.concatenate((folds[0:(lengths[0]+lengths[1])], folds[(lengths[0]+lengths[1]+lengths[2]):]))
    elif ts == 3:
        # 4th training split as test
        y_test = big_mac_y[(lengths[0]+lengths[1]+lengths[2]):(lengths[0]+lengths[1]+lengths[2]+lengths[3])]
        X_test = big_mac[(lengths[0]+lengths[1]+lengths[2]):(lengths[0]+lengths[1]+lengths[2]+lengths[3])]
        big_mac = np.concatenate((big_mac[0:(lengths[0]+lengths[1]+lengths[2])],
                                  big_mac[(lengths[0]+lengths[1]+lengths[2]+lengths[3]):]))
        big_mac_y = np.concatenate((big_mac_y[0:(lengths[0]+lengths[1]+lengths[2])],
                                    big_mac_y[(lengths[0]+lengths[1]+lengths[2]+lengths[3]):]))

        folds = np.concatenate((folds[0:(lengths[0]+lengths[1]+lengths[2])],
                                folds[(lengths[0]+lengths[1]+lengths[2]+lengths[3]):]))
    elif ts == 4:
        # 5th training split as test
        y_test = big_mac_y[(lengths[0]+lengths[1]+lengths[2]+lengths[3]):]
        X_test = big_mac[(lengths[0]+lengths[1]+lengths[2]+lengths[3]):]
        big_mac = big_mac[0:(lengths[0]+lengths[1]+lengths[2]+lengths[3])]
        big_mac_y = big_mac_y[0:(lengths[0]+lengths[1]+lengths[2]+lengths[3])]

        folds = folds[0:(lengths[0]+lengths[1]+lengths[2]+lengths[3])]
    if ts > 0:
        old_index = folds[0]
        x = 0
        for i in range(0, len(folds)):
            if old_index != folds[i]:
                # new fold
                x += 1
                old_index = folds[i]
            folds[i] = x
    print(folds)
    new_train_set_size = len(folds)
    print(str(len(folds)))
    assert (old_train_set_size - new_train_set_size) == len(X_test)

In [ ]:
ps = PredefinedSplit(folds)
print("n splits: " + str(ps.get_n_splits()))

for train_index, val_index in ps.split():
     print("TRAIN:", train_index, "TEST:", val_index)
     X_train, X_val = big_mac[train_index], big_mac[val_index]
     y_train, y_val = big_mac_y[train_index], big_mac_y[val_index]

In [ ]:
mlp = MLPClassifier(activation='relu', solver='adam', early_stopping=True, learning_rate='invscaling',
                    hidden_layer_sizes=(80,),
                    alpha=0.001,
                    #learning_rate_init=0.00075,
                    max_iter=50,
                    random_state=42,
                   )

rfc = RandomForestClassifier(n_jobs=-1,
                             # n_estimators=70,
                             # class_weight='balanced_subsample'
                            )

linsvc = LinearSVC(max_iter=1000, random_state=0)


cv = StratifiedKFold(n_splits=4) # only for testing

# param grid: add (lowercase) typename__ in front of every hyperparameter
params = {# MLP
          #'mlpclassifier__hidden_layer_sizes': [(80,),],  # mlpclassifier__
          #'mlpclassifier__max_iter': [50,],
          #'mlpclassifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
          #'mlpclassifier__learning_rate_init': [0.001, 0.0005],
          #'mlpclassifier__alpha': [0.0001,0.001],
          # RFC
          #'randomforestclassifier__max_depth': [20, 30],  # randomforestclassifier__
          #'randomforestclassifier__n_estimators': [70, 90],
          #'class_weight': ["balanced", "balanced_subsample", "None"],
          # SVM
          #'C': [1.0, 4.0 ],  # linearsvc__
          # RUS
          'randomundersampler__sampling_strategy': [0.2],
          }

# imba_pipeline = make_pipeline(SMOTE(random_state=42, sampling_strategy=0.1, n_jobs=-1), mlp)
imba_pipeline = make_pipeline(RandomUnderSampler(random_state=42), mlp)
# to try: sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)

# Perform grid search
mcc_scorer = make_scorer(matthews_corrcoef)
gs = GridSearchCV(estimator=imba_pipeline, cv=ps, param_grid=params,
                  scoring=mcc_scorer, return_train_score=True)

In [ ]:
print("*TRAINING*\t" + str(datetime.now()))

# gs.fit(big_mac, big_mac_y)
gs.fit(all_X_train[2], all_y_train[2])
best_score = round(gs.best_score_, 5)

# print results
print("    Classifier:  " + str(gs.estimator))
print("    Best score:  %f" % best_score)
print("    Best params: %s" % str(gs.best_params_))
cv_results = pd.DataFrame(gs.cv_results_)
print(f"   \nAll training results:\n{cv_results}")

In [ ]:
# pickle_file_name = "gs_" + msa_type + "_" + str(best_score).replace(".", "-") + ".pickle"
# with open(("../data/pickles/" + pickle_file_name), "wb") as pickle_file:
#    pickle.dump(gs, pickle_file)

In [ ]:
print("*TESTING*\t" + str(datetime.now()))

best_classifier = gs.best_estimator_
print(best_classifier)
y_pred = best_classifier.predict(X_test)
y_proba = gs.predict_proba(X_test).transpose()[1].transpose()  # assumes 2nd column corresponds to class 1
pred_score = best_classifier.score(X_test, y_test)

In [ ]:
print("*RESULTS*\t" + str(datetime.now()))

cm = confusion_matrix(y_test, y_pred)
print(cm)
perf_metrics = [0]*7
perf_metrics[0] = matthews_corrcoef(y_test, y_pred)
perf_metrics[1] = accuracy_score(y_test, y_pred)
perf_metrics[2] = precision_score(y_test, y_pred)
perf_metrics[3] = recall_score(y_test, y_pred)
perf_metrics[4] = roc_auc_score(y_true=y_test, y_score=y_proba)
perf_metrics[5] = balanced_accuracy_score(y_test, y_pred)
perf_metrics[6] = f1_score(y_test, y_pred)
# print("    Pred:        " + str(pred_score))  # should be MCC but is accuracy?
print("    MCC:         " + str(perf_metrics[0]))
print("    Accuracy:    " + str(perf_metrics[1]))
print("    Precision:   " + str(perf_metrics[2]))
print("    Recall:      " + str(perf_metrics[3]))
print("    AUC-ROC:     " + str(perf_metrics[4]))
print("    BalancedAcc: " + str(perf_metrics[5]))
print("    F1 Score:    " + str(perf_metrics[6]))

In [ ]:
print("Precision-Recall curve:")
prec_score, rec_score, thresholds_pr = precision_recall_curve(y_true=y_test, probas_pred=y_proba)
av_prec_score = average_precision_score(y_true=y_test, y_score=y_proba)
disp = plot_precision_recall_curve(gs, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(av_prec_score))

print("ROC curve:")
fpr, tpr, thresholds_roc = roc_curve(y_true=y_test, y_score=y_proba)
disp2 = plot_roc_curve(gs, X_test, y_test)
disp2.ax_.set_title('2-class ROC curve: '
                   'AUC={0:0.2f}'.format(perf_metrics[4]))
plt_roc.show()

print("Confusion matrices:")
class_names = ['non_binding', 'binding']
plot_confusion_matrix(gs, X_test, y_test, display_labels=class_names)
plt_confm.show()
plot_confusion_matrix(gs, X_test, y_test, display_labels=class_names,
                     normalize='true')
plt_confm_n.show()

In [ ]:
standard_errors = calculate_se(json_file, embeddings_file, y_pred, y_test)


# remove ROC AUC (no SE) from metrics
del perf_metrics[4]

print("Confidence intervals and standard errors:")
conf_intervals = [0]*len(standard_errors)
for i in range(0, len(perf_metrics)):
    # CI: use student's t-distriution for 195-1 = 194 = ~200 DOF and conf level = 0.95 -> 1.972
    # formula: mean +- t*SE
    conf_intervals[i] = 1.972 * standard_errors[i]
    print("\t" + str(round(perf_metrics[i], 5)) + "\t +/- " + str(conf_intervals[i]))

In [ ]:
print("Comparison to baseline:")
n = len(y_test)
probabilities = np.array([0.9185, 0.0815])
y_pred_random = np.zeros(shape=n)
for i in range(0, n):
    y_pred_random[i] = np.random.choice(a=[0, 1], p=probabilities)

y_pred_majclass = np.zeros(shape=len(y_test))
print("    Random:")
print("\tMCC:         " + str(matthews_corrcoef(y_test, y_pred_random)))
print("\tAccuracy:    " + str(accuracy_score(y_test, y_pred_random)))
print("\tPrecision:   " + str(precision_score(y_test, y_pred_random)))
print("\tRecall:      " + str(recall_score(y_test, y_pred_random)))
print("\tBalancedAcc: " + str(balanced_accuracy_score(y_test, y_pred_random)))
print("\tF1 score:    " + str(f1_score(y_test, y_pred_random)))
print("    Majority class [0]:")
print("\tMCC:         " + str(matthews_corrcoef(y_test, y_pred_majclass)))
print("\tAccuracy:    " + str(accuracy_score(y_test, y_pred_majclass)))
print("\tPrecision:   " + str(precision_score(y_test, y_pred_majclass)))
print("\tRecall:      " + str(recall_score(y_test, y_pred_majclass)))
print("\tBalancedAcc: " + str(balanced_accuracy_score(y_test, y_pred_majclass)))
print("\tF1 score:    " + str(f1_score(y_test, y_pred_majclass)))
